In [ ]:
import geopandas
import fiona

print('Importing bases...')
filepath = './data/CAD_BATI3D_BASE.gml'
bases = geopandas.read_file(filepath)
print('Converting bases to espg4326')
bases = bases.set_crs(crs=None, epsg=2056)
bases = bases.to_crs(crs=None, epsg=4326)

"""print('Importing facades...')
filepath = './data/CAD_BATI3D_BASIC_FACADE.gml'
facades = geopandas.read_file(filepath)
print('Converting facades to espg4326')
à = facades.set_crs(crs=None, epsg=2056)
facades = facades.to_crs(crs=None, epsg=4326)
"""

print('Importing toits...')
filepath = './data/CAD_BATI3D_BASIC_TOIT.gml'
toits = geopandas.read_file(filepath)
print('Converting toits to espg4326')
toits = toits.set_crs(crs=None, epsg=2056)
toits = toits.to_crs(crs=None, epsg=4326)

In [ ]:
bases['coordinates'] = [list(bases['geometry'][i].geoms[0].exterior.coords) for i in range(0,len(bases))]
bases['2D_coordinates'] = [np.array(list(bases['geometry'][i].geoms[0].exterior.coords))[:,:2].tolist()for i in range(0,len(bases))]
bases['base_z'] = [np.mean(np.array(list(bases['geometry'][i].geoms[0].exterior.coords))[:,1])for i in range(0,len(bases))]
data = bases.merge(toits, how='inner', on='EGID', suffixes=('_bases', '_toit'))
data['hauteur'] = data['ALTITUDE_MAX'] - data['base_z']

In [ ]:
data

## no elevation

In [ ]:
import pydeck

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=46.1966,
  longitude=6.09,
  zoom=11,
  max_zoom=50,
  pitch=45,
  bearing=0
)

bases_layer = pydeck.Layer('PolygonLayer',
                           data=bases,
                           opacity=0.8,
                           stroked=False,
                           get_polygon="2D_coordinates",
                           filled=True,
                           extruded=True,
                           wireframe=True,
                           get_elevation='hauteur',
                           get_fill_color=[255, 0, 255],
                           get_line_color=[255, 255, 255],
                           auto_highlight=True,
                           pickable=True)

r = pydeck.Deck(
    layers=[bases_layer],
    initial_view_state=INITIAL_VIEW_STATE)

# r.show()
r.to_html()

### elevation

In [ ]:
import pydeck

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=46.1966,
  longitude=6.09,
  zoom=11,
  max_zoom=32,
  pitch=45,
  bearing=0
)

bases_layer = pydeck.Layer('PolygonLayer',
                           data=bases,
                           opacity=0.8,
                           stroked=False,
                           get_polygon="coordinates",
                           filled=True,
                           extruded=True,
                           wireframe=True,
                           get_elevation='ALTITUDE_MAX',
                           get_fill_color=[255, 0, 255],
                           get_line_color=[255, 255, 255],
                           auto_highlight=True,
                           pickable=True)

r = pydeck.Deck(
    layers=[bases_layer],
    initial_view_state=INITIAL_VIEW_STATE)

# r.show()
r.to_html()